In [1]:
# Teachable machine quick demo!
# https://teachablemachine.withgoogle.com/train
# motivation credit - https://stackoverflow.com/questions/71816488/model-is-not-giving-correct-results-even-if-accuracy-is-good

## Train the model, parameters
<img src="Model Training_ TM.png">

## download as keras
<img src="TM_save as keras.png">

## download as Tensorflow lite
<img src="TM_save_As_TFlite.png">

In [2]:
#import the libs
import tensorflow as tf
from PIL import Image, ImageOps
import numpy as np
import pandas as pd

In [3]:
#Load label and model
#build label dictionary
with open("converted_keras/labels.txt") as fl:
    label_lines=fl.read().splitlines()
labels={} # "0" --> A, "1"-->"B"
for each_l in label_lines:
    labels[each_l.split()[0]]=each_l.split()[1]

#load the model
model=tf.keras.models.load_model('converted_keras/keras_model.h5')

2022-04-11 13:59:48.472582: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Predict the label
# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
# Replace this with the path to your image
image = Image.open('Screen Shot 2022-04-11 at 10.31.46 AM.png').convert('RGB')# convert screen snap to RGB
#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)
# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
# Load the image into the array
data[0] = normalized_image_array

# run the inference
prediction = model.predict(data)
#print(prediction)
print("Predicted label : ",labels[str(np.argmax(prediction[0]))])

2022-04-11 13:59:53.149889: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Predicted label :  P


In [5]:
#now with converted_tflite
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="converted_tflite/model_unquant.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


interpreter.invoke()


In [6]:
img = Image.open('Screen Shot 2022-04-11 at 10.31.46 AM.png').convert("RGB")
img = ImageOps.fit(img, size, Image.ANTIALIAS)
input_data = np.expand_dims(img, axis=0).astype(np.float32)
input_data /= 255.
input_data
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
#print(np.argmax(output_data[0]))
print("Predicted label : ",labels[str(np.argmax(output_data[0]))])

Predicted label :  P


In [8]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 1280)              410208    
_________________________________________________________________
sequential_3 (Sequential)    (None, 29)                131000    
Total params: 541,208
Trainable params: 527,128
Non-trainable params: 14,080
_________________________________________________________________
